<a href="https://colab.research.google.com/github/wenyenluan/line-bot-tutorial/blob/master/SalesForecasting_gbdt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
from time import time
from datetime import date, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import random
pd.set_option('display.max_columns', 500)

# 1 預處理

In [0]:
"""
train_data_path = "train.csv"
df_train = pd.read_csv(
     train_data_path, usecols=[1, 2, 3, 4, 5], dtype={'onpromotion': bool}, 
     converters={'unit_sales': lambda x: np.log1p(float(x)) if float(x) > 0 else 0}, 
     parse_dates=['date'], skiprows=range(1, 66458909)
 )
df_2017 = df_train[df_train['date'] >= '2017-01-01']
"""

In [0]:
#下載所需資料
!wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=18cOyvZjZXwnwMnemZkkz5uiNfOU_N2Jd' -O df_2017.csv.zip

!wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1KwQOWkn4KP5Sp51pVKev9ZsK1KNs9K1W' -O items.csv

!wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1PsAVhC48ELirP_2VyBKrz52dJgcXXv18' -O stores.csv

!wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1_tINBIYsOLW25RAVmMerwg2M1dPzHiAC' -O test.csv.zip


will be placed in the single file you specified.

--2019-03-25 15:20:08--  https://docs.google.com/uc?export=download&id=18cOyvZjZXwnwMnemZkkz5uiNfOU_N2Jd
Resolving docs.google.com (docs.google.com)... 74.125.141.100, 74.125.141.139, 74.125.141.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.141.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘df_2017.csv.zip’

df_2017.csv.zip         [ <=>                ]   3.17K  --.-KB/s    in 0s      

2019-03-25 15:20:08 (40.7 MB/s) - ‘df_2017.csv.zip’ saved [3251]

Loading robots.txt; please ignore errors.
--2019-03-25 15:20:08--  https://docs.google.com/robots.txt
Reusing existing connection to docs.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘df_2017.csv.zip’

df_2017.csv.zip         [ <=>                ]     567  --.-KB/s    in 0s      

2019-03-25 15:20:09 (58.3 MB/s) - ‘df_2017.csv.zip’ saved [567]

In [0]:
#讀取csv檔案
items = pd.read_csv("items.csv").set_index('item_nbr')
stores = pd.read_csv("stores.csv").set_index("store_nbr")
df_test = pd.read_csv("test.csv.zip",  parse_dates=['date'] )
df_2017 = pd.read_csv("df_2017.csv.zip", parse_dates=['date']) #讀取需要一段時簡

In [0]:
store_numbers = list(df_2017['store_nbr'].unique())

#抽4間店的資料
sample_size = 4 

sorted_sample = [
    store_numbers[i] for i in sorted(random.sample(range(len(store_numbers)), sample_size))
]
print (sorted_sample)
print (df_2017.shape)

#抽樣幾家店的銷售資料
df_2017 = df_2017[df_2017['store_nbr'].isin(sorted_sample)]

In [0]:
df_2017.head()

,Unnamed: 0,date,store_nbr,item_nbr,unit_sales,onpromotion
50000,35279871,2017-01-02,24,103665,1.098612,False
50001,35279872,2017-01-02,24,105574,1.945910,False
50002,35279873,2017-01-02,24,105575,2.302585,False
50003,35279874,2017-01-02,24,105693,1.098612,False
50004,35279875,2017-01-02,24,105737,1.609438,False


In [0]:


df_test = pd.read_csv(
    "test.csv.zip", sep=',', # usecols=[0, 1, 2, 3, 4], 
    dtype={'onpromotion': bool}, parse_dates=['date']
)


#items = pd.read_csv("items.csv").set_index('item_nbr')



In [0]:

df_test = df_test[df_test['store_nbr'].isin(sorted_sample)]
df_test.set_index(['store_nbr', 'item_nbr', 'date'], inplace=True)

![alt text](https://cdn-images-1.medium.com/max/1000/1*fiaK_Nh_vMSDX2R019S55g.png)

In [0]:
encoder = LabelEncoder()

items['family'] = encoder.fit_transform(items['family'].values)

stores['city'] = encoder.fit_transform(stores['city'].values)
stores['state'] = encoder.fit_transform(stores['state'].values)
stores['type'] = encoder.fit_transform(stores['type'].values)

每個店鋪每件商品每天的促銷情況

![alt text](https://cdn-images-1.medium.com/max/1000/1*CKVCgk7z_F_wulgrorFxRQ.png)

In [0]:
promo_2017_train = df_2017.set_index(
    ['store_nbr', 'item_nbr', 'date'])[['onpromotion']].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)

promo_2017_test = df_test[['onpromotion']].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)

promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)

del promo_2017_train, promo_2017_test
gc.collect()

159

### 每個店鋪每件商品每天的銷量

In [0]:
df_2017 = df_2017.set_index(
    ['store_nbr', 'item_nbr', 'date'])[['unit_sales']].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

![alt text](https://cdn-images-1.medium.com/max/1000/1*WuM4X_LALBGn8lqYjcs08g.png)

In [0]:
items = items.reindex(df_2017.index.get_level_values(1))
stores = stores.reindex(df_2017.index.get_level_values(0))

### 每件商品每天的銷量

In [0]:
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()

![alt text](https://cdn-images-1.medium.com/max/1000/1*O3_a8eBE7YIyrWm9Iy9xHA.png)

### 每件商品每天的促銷情況

In [0]:
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

### 每個種類的商品在每個店鋪每天的銷量

In [0]:
df_2017_store_class = df_2017.reset_index()
df_2017_store_class['class'] = items['class'].values
df_2017_store_class_index = df_2017_store_class[['class', 'store_nbr']]
df_2017_store_class = df_2017_store_class.groupby(['class', 'store_nbr'])[df_2017.columns].sum()

![alt text](https://cdn-images-1.medium.com/max/1000/1*Mx-uHn-TWtdkPL4z7nHciQ.png)

### 每個種類的商品在每個店舖每天的促銷情況

In [0]:
promo_2017_store_class = promo_2017.reset_index()
promo_2017_store_class['class'] = items['class'].values
promo_2017_store_class_index = promo_2017_store_class[['class', 'store_nbr']]
promo_2017_store_class = promo_2017_store_class.groupby(['class', 'store_nbr'])[promo_2017.columns].sum()

![alt text](https://cdn-images-1.medium.com/max/1000/1*rOs_knvENM8t1h9wYb7nKg.png)

# 2 特徵工程（準備數據集）

In [0]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [0]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    # 促銷天數特徵（6個特徵）
    X = {
        'promo_14_2017': get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,  # 前14天中每個店鋪商品的促銷天數
        'promo_60_2017': get_timespan(promo_df, t2017, 60, 60).sum(axis=1).values,  # 前60天中每個店鋪商品的促銷天數
        'promo_140_2017': get_timespan(promo_df, t2017, 140, 140).sum(axis=1).values,  # 前140天中每個店鋪商品的促銷天數
        'promo_3_2017_aft': get_timespan(promo_df, t2017 + timedelta(days=16), 15, 3).sum(axis=1).values,  # 後3天中每個店鋪商品的促銷天數
        'promo_7_2017_aft': get_timespan(promo_df, t2017 + timedelta(days=16), 15, 7).sum(axis=1).values,  # 後7天中每個店鋪商品的促銷天數
        'promo_14_2017_aft': get_timespan(promo_df, t2017 + timedelta(days=16), 15, 14).sum(axis=1).values  # 後14天中每個店鋪商品的促銷天數
    }
    
    # 促銷銷量和正常銷量特徵（24個特徵）
    for i in [3, 7, 14, 30, 60, 140]:
        tmp1 = get_timespan(df, t2017, i, i)  # 前i天每個店鋪商品的銷量
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1  # 前i天每個店鋪商品的促銷情況（是否促銷）
        
        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values  # 前i天每個店鋪商品平均促銷銷量
        X['has_promo_mean_%s_decay' % i] = (tmp1 * tmp2.replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每個店鋪商品促銷銷量和（带衰减）
        
        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values  # 前i天每個店鋪商品平均正常銷量
        X['no_promo_mean_%s_decay' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan) * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每個店鋪商品正常銷量和（带衰减）
        
    # 銷量統計特徵（42個特徵）
    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)  # 前i天每個店鋪商品的銷量
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i天每個店鋪商品銷量的平均一阶差分
        X['mean_%s_decay' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i天每個店鋪商品銷量的和（带衰减）
        X['mean_%s' % i] = tmp.mean(axis=1).values  # 前i天每個店鋪商品銷量的平均值
        X['median_%s' % i] = tmp.median(axis=1).values  # 前i天每個店鋪商品銷量的中位数
        X['min_%s' % i] = tmp.min(axis=1).values  # 前i天每個店鋪商品銷量的最小值
        X['max_%s' % i] = tmp.max(axis=1).values  # 前i天每個店鋪商品銷量的最大值
        X['std_%s' % i] = tmp.std(axis=1).values  # 前i天每個店鋪商品銷量的標準差
    
    # 銷量統計特徵2（42個特徵）
    for i in [3, 7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017 + timedelta(days=-7), i, i)  # 7天前的前i天每個店鋪商品的銷量
        X['diff_%s_mean_2' % i] = tmp.diff(axis=1).mean(axis=1).values  # 7天前的前i天每個店鋪商品銷量的平均一階差分
        X['mean_%s_decay_2' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 7天前的前i天每個店鋪商品銷量的和（带衰减）
        X['mean_%s_2' % i] = tmp.mean(axis=1).values  # 7天前的前i天每個店鋪商品銷量的平均值
        X['median_%s_2' % i] = tmp.median(axis=1).values  # 7天前的前i天每個店鋪商品銷量的中位数
        X['min_%s_2' % i] = tmp.min(axis=1).values  # 7天前的前i天每個店鋪商品銷量的最小值
        X['max_%s_2' % i] = tmp.max(axis=1).values  # 7天前的前i天每個店鋪商品銷量的最大值
        X['std_%s_2' % i] = tmp.std(axis=1).values  # 7天前的前i天每個店鋪商品銷量的標準差
        
    # 有銷量和有促銷的天數特徵（30個特徵）
    for i in [7, 14, 30, 60, 140]:
        tmp = get_timespan(df, t2017, i, i)  # 前i天每個店鋪商品的銷量
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i天每個店鋪商品有銷量的天數
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i天每個店鋪商品距離上一次有銷量的天數
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i天每個店鋪商品距離第一次有銷量的天數
        
        tmp = get_timespan(promo_df, t2017, i, i)  # 前i天每個店鋪商品的促銷情況
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i天每個店鋪商品有促銷的天數
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i天每個店鋪商品距離上一次有促銷的天數
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i天每個店鋪商品距離第一次有促銷的天數
        
    # 後15天的促銷情況特徵（3個特徵）
    tmp = get_timespan(promo_df, t2017 + timedelta(days=16), 15, 15)  # 後15天每個店鋪商品的促銷情況
    X['has_promo_days_in_after_15_days'] = (tmp > 0).sum(axis=1).values  # 後15天每個店鋪商品有促銷的天數
    X['last_has_promo_day_in_after_15_days'] = ((tmp > 0) * np.arange(1, 16)).max(axis=1).values  # 後15天每個店鋪商品距離最後一次有促銷的天數
    X['first_has_promo_day_in_after_15_days'] = 16 - ((tmp > 0) * np.arange(15, 0, -1)).max(axis=1).values  # 後15天每個店鋪商品距離最近一次有促銷的天數
    
    # 前15天的銷量（15個特徵）
    for i in range(1, 16):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()
    
    # 前4（20）周每天的平均銷量（14個特徵）
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values
        
    # 前16天到後15天每天的促銷情況（32個特徵）
    for i in range(-16, 16):
        X['promo_{}'.format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)
        
    X = pd.DataFrame(X)
    
    if is_train:
        y = df[pd.date_range(t2017, periods=16)].values
        return X, y
    
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    
    return X

# 2.1 資料集的切分概念介紹
![alt text](https://developers.google.com/machine-learning/crash-course/images/WorkflowWithValidationSet.svg)

## 2.2 準備訓練集 (Training Set)

In [0]:
print("[INFO] Preparing training data...")

t2017 = date(2017, 6, 14)
num_days = 6
X_l, y_l = [], []
for i in range(num_days):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)
    
    X_tmp2 = prepare_dataset(df_2017_item, promo_2017_item, t2017 + delta, is_train=False, name_prefix='item')
    X_tmp2.index = df_2017_item.index
    X_tmp2 = X_tmp2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)
    
    X_tmp3 = prepare_dataset(df_2017_store_class, promo_2017_store_class, t2017 + delta, is_train=False, name_prefix='store_class')
    X_tmp3.index = df_2017_store_class.index
    X_tmp3 = X_tmp3.reindex(df_2017_store_class_index).reset_index(drop=True)
    
    X_tmp = pd.concat([X_tmp, X_tmp2, X_tmp3, items.reset_index(), stores.reset_index()], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
    
    del X_tmp, y_tmp, X_tmp2, X_tmp3
    gc.collect()

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l
gc.collect()

[INFO] Preparing training data...


0

In [0]:
np.save('X_train.npy', X_train) 
np.save('y_train.npy', y_train) 

## 2.3 準備驗證集 (Validation Set)

In [0]:
print("[INFO] Preparing validation data...")

X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 26))

X_val2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 7, 26), is_train=False, name_prefix='item')
X_val2.index = df_2017_item.index
X_val2 = X_val2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_val3 = prepare_dataset(df_2017_store_class, promo_2017_store_class, date(2017, 7, 26), is_train=False, name_prefix='store_class')
X_val3.index = df_2017_store_class.index
X_val3 = X_val3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_val = pd.concat([X_val, X_val2, X_val3, items.reset_index(), stores.reset_index()], axis=1)

del X_val2, X_val3
gc.collect()

[INFO] Preparing validation data...


21

In [0]:
np.save('X_val.npy', X_val) 
np.save('y_val.npy', y_val) 

## 2.4 準備測試集 (Test Set)

In [0]:
print("[INFO] Preparing testing data...")

X_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 16), is_train=False)

X_test2 = prepare_dataset(df_2017_item, promo_2017_item, date(2017, 8, 16), is_train=False, name_prefix='item')
X_test2.index = df_2017_item.index
X_test2 = X_test2.reindex(df_2017.index.get_level_values(1)).reset_index(drop=True)

X_test3 = prepare_dataset(df_2017_store_class, promo_2017_store_class, date(2017, 8, 16), is_train=False, name_prefix='store_class')
X_test3.index = df_2017_store_class.index
X_test3 = X_test3.reindex(df_2017_store_class_index).reset_index(drop=True)

X_test = pd.concat([X_test, X_test2, X_test3, items.reset_index(), stores.reset_index()], axis=1)

del X_test2, X_test3
gc.collect()

[INFO] Preparing testing data...


21

In [0]:
np.save('X_test.npy', X_test) 


# 3 訓練模型 (可能要執行10分鐘左右)

In [0]:
print("[INFO] Start training...")
t0 = time()

params = {
    'num_leaves': 80, 
    'objective': 'regression', 
    'min_data_in_leaf': 200, 
    'learning_rate': 0.02, 
    'feature_fraction': 0.8, 
    'bagging_fraction': 0.7, 
    'bagging_freq': 1, 
    'metric': 'l2', 
    'num_threads': 16
}

MAX_ROUNDS = 50
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print('=' * 50)
    print("Step %d" % (i + 1))
    print('=' * 50)
    
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i], 
        categorical_feature=cate_vars, 
        weight=pd.concat([items['perishable']] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain, 
        categorical_feature=cate_vars, 
        weight=items['perishable'] * 0.25 + 1
    )
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS, 
        valid_sets=[dtrain, dval], early_stopping_rounds=125, 
        verbose_eval=50
    )
    print('\n'.join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance('gain')), key=lambda x: x[1], reverse=True)))
    val_pred.append(
        bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(
        bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("[INFO] Done in %f seconds." % (time() - t0))
print("[INFO] Training Finished! ( ^ _ ^ ) V")
    
print("Validation mse:", mean_squared_error(y_val, np.array(val_pred).transpose()))

weight = items['perishable'] * 0.25 + 1
err = (y_val - np.array(val_pred).transpose()) ** 2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print("nwrmsle = {}".format(err))

y_val = np.array(val_pred).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index, 
    columns=pd.date_range('2017-07-26', periods=16)
).stack().to_frame('unit_sales')
df_preds.index.set_names(['store_nbr', 'item_nbr', 'date'], inplace=True)
df_preds['unit_sales'] = np.clip(np.expm1(df_preds['unit_sales']), 0, 1000)
df_preds.reset_index().to_csv('lgb_cv.csv', index=False)

[INFO] Start training...
Step 1


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 0.422211	valid_1's l2: 0.429549
Did not meet early stopping. Best iteration is:
[50]	training's l2: 0.422211	valid_1's l2: 0.429549
mean_60_decay: 391450.48
mean_140_decay: 343837.55
mean_14_decay: 91488.42
mean_30_decay: 86654.87
mean_7_decay: 83252.83
mean_7: 43344.61
mean_20_dow0_2017: 42996.15
promo_0: 39264.55
no_promo_mean_140_decay: 7330.87
mean_4_dow0_2017: 6993.55
no_promo_mean_60_decay: 5743.07
item_promo_0: 5634.10
median_7: 4380.41
no_promo_mean_14_decay: 4213.37
day_1_2017: 3943.75
has_promo_mean_140: 3098.87
no_promo_mean_140: 2845.44
has_promo_mean_60: 2593.64
no_promo_mean_7_decay: 2241.20
no_promo_mean_30_decay: 2185.36
has_promo_mean_30: 2071.56
no_promo_mean_60: 1895.91
mean_30: 1358.39
last_has_sales_day_in_last_14: 1255.79
day_7_2017: 1188.93
no_promo_mean_3_decay: 1074.56
no_promo_mean_30: 976.61
no_promo_mean_3: 830.25
has_promo_mean_14: 827.79
day_14_2017: 783.16
std_30: 758.37
it

# 4 針對測試集做預測

### 測試集日期區間

![alt text](https://cdn-images-1.medium.com/max/1000/1*KeZLDP3aW89LFFIa1c8-Rg.png)

In [0]:
print("[INFO] Making submission...")

y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index, 
    columns=pd.date_range('2017-08-16', periods=16)
).stack().to_frame('unit_sales')
df_preds.index.set_names(['store_nbr', 'item_nbr', 'date'], inplace=True)

[INFO] Making submission...


In [0]:
submission = df_test[['id']].join(df_preds, how='left').fillna(0)
submission['unit_sales'] = np.clip(np.expm1(submission['unit_sales']), 0, 1000)
submission.to_csv('lgb_sub.csv', float_format='%.4f', index=None)

In [0]:
submission.head(30)

id  unit_sales
store_nbr item_nbr date                             
24        96995    2017-08-16  125586763    0.000000
          99197    2017-08-16  125586764    0.589750
          103501   2017-08-16  125586765    0.000000
          103520   2017-08-16  125586766    2.314444
          103665   2017-08-16  125586767    1.970807
          105574   2017-08-16  125586768    5.503547
          105575   2017-08-16  125586769    3.708123
          105576   2017-08-16  125586770    0.000000
          105577   2017-08-16  125586771    0.831046
          105693   2017-08-16  125586772    0.872365
          105737   2017-08-16  125586773    4.853985
          105857   2017-08-16  125586774    1.330910
          106716   2017-08-16  125586775    0.606835
          108079   2017-08-16  125586776    0.953157
          108634   2017-08-16  125586777    0.485217
          108696   2017-08-16  125586778    1.624500
          108698   2017-08-16  125586779    2.371395
          108701   2017-08-16  125586780    1.642820
          108786   2017-08-16  125586781    0.909950
          108797   2017-08-16  125586782    3.869679
          108831   2017-08-16  125586783    2.615782
          108833   2017-08-16  125586784    0.000000
          108862   2017-08-16  125586785    1.051203
          108952   2017-08-16  125586786    2.216111
          111223   2017-08-16  125586787    1.974222
          111397   2017-08-16  125586788    4.092080
          112830   2017-08-16  125586789    0.885840
          114778   2017-08-16  125586790    2.969636
          114790   2017-08-16  125586791    4.277350
          114799   2017-08-16  125586792    0.898796